# Model Iteration 1
Here, I want to implement a very simple model. To do this, I'm not going to create very many features, just recode some of the variables that are categorical like district. 

## Importing Everything!

In [11]:
% matplotlib inline

import shapefile
import pandas as pd
import numpy as np
import itertools

import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib import cm
from datetime import datetime
from ipywidgets import widgets  
from IPython.display import display

from sklearn import cross_validation
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import KFold
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.cross_validation import train_test_split
from sklearn.datasets import load_iris
from sklearn import tree
from sklearn.metrics import log_loss


## Loading in the data
Now, I need to load in the data

In [2]:
crimeData = pd.read_csv('train.csv')

## Helper functions for recoding data
Here are the helper functions for recoding data. We'll add more as we create some new features

In [3]:
def recodeData(df, isTrain = False):
    '''This function takes in the dataframe that we get from loading in the 
    SF crime data and returns a re-coded dataframe that has all the 
    additional features we want to add and the categorical features recoded 
    and cleaned.
    '''

    #since the modifications are done in-place we don't return the dataframe. 
    #we do, however, return the list of all the columns we added.
    newDate = recodeDates(df)
    newDistrict = recodePoliceDistricts(df)
    
    addedColumns = [] 
    addedColumns += newDate
    addedColumns += newDistrict 

    if (isTrain):
        try: #prevents error if the coumns have already been removed
            columnsToDrop = ['Descript', 'Resolution']
            df.drop(columnsToDrop, axis=1, inplace=True)
        except:
            print "already recoded"

    return df, addedColumns

In [4]:
def recodeDates(df):
    '''This function takes in a dataframe and recodes the date field into 
    useable values. Here, we also recode the day of week.'''
    #Recode the dates column to year, month, day and hour columns
    df['DateTime'] = df['Dates'].apply(
        lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

    df['Year'] = df['DateTime'].apply(lambda x: x.year)
    df['Month'] = df['DateTime'].apply(lambda x: x.month)
    df['Day'] = df['DateTime'].apply(lambda x: x.day)
    df['Hour'] = df['DateTime'].apply(lambda x: x.hour)
    df['Minute'] = df['DateTime'].apply(lambda x: x.minute)
    df['DayOfWeekRecode'] = df['DateTime'].apply(lambda x: x.weekday())
    df['MinuteOfWeek'] = df['DateTime'].apply(lambda x: x.weekday()*24*60 + x.hour*60 + x.minute)

    return ['Year', 'Month', 'Day', 'Hour', 'Minute', 'DayOfWeekRecode', 'MinuteOfWeek']

In [5]:
def recodePoliceDistricts(df):
    '''This function recodes the police district to a one-hot encoding 
    scheme.'''
    districts = df['PdDistrict'].unique().tolist()
    newColumns = []
    for district in districts:
        newColumns.append('District' + district)
        df['District' + district] = df['PdDistrict'].apply(
            lambda x: int(x == district))

    return newColumns

## Recoding Columns
Here, I want to do some recoding of the columns. To do this, I'm going to use our handy-dandy helper function.  

In [6]:
crimeData, addedColumns = recodeData(
    crimeData, isTrain = True)
crimeData.describe()

,X,Y,Year,Month,Day,Hour,Minute,DayOfWeekRecode,MinuteOfWeek,DistrictNORTHERN,DistrictPARK,DistrictINGLESIDE,DistrictBAYVIEW,DistrictRICHMOND,DistrictCENTRAL,DistrictTARAVAL,DistrictTENDERLOIN,DistrictMISSION,DistrictSOUTHERN
count,878049.000000,878049.000000,878049.000000,878049.000000,878049.000000,878049.000000,878049.000000,878049.000000,878049.000000,878049.000000,878049.000000,878049.000000,878049.000000,878049.000000,878049.000000,878049.000000,878049.000000,878049.000000,878049.000000
mean,-122.422616,37.771020,2008.712046,6.436509,15.570623,13.412655,20.155026,2.992691,5134.388787,0.119920,0.056162,0.089796,0.101852,0.051488,0.097329,0.074707,0.093171,0.136562,0.179013
std,0.030354,0.456893,3.631194,3.428972,8.783005,6.549573,18.594915,1.972023,2858.378361,0.324869,0.230234,0.285889,0.302454,0.220991,0.296406,0.262917,0.290673,0.343384,0.383363
min,-122.513642,37.707879,2003.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-122.432952,37.752427,2006.000000,3.000000,8.000000,9.000000,0.000000,1.000000,2600.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,-122.416420,37.775421,2009.000000,6.000000,16.000000,14.000000,19.000000,3.000000,5220.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,-122.406959,37.784369,2012.000000,9.000000,23.000000,19.000000,33.000000,5.000000,7606.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,-120.500000,90.000000,2015.000000,12.000000,31.000000,23.000000,59.000000,6.000000,10079.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Model Iteration 1
Now that I've done some recoding, I'm going to create my model. To do this, I'm going to do a random forest classifier. 

In [15]:
columnsToUse = addedColumns + ['X','Y']

X = crimeData[columnsToUse]
y = crimeData['Category']

clf = RandomForestClassifier(n_estimators=10, max_depth=None, min_samples_split=1, random_state=0)

k_folds = StratifiedShuffleSplit(y, 3, test_size=0.5, random_state=0)

scores = []

for k, (train, test) in enumerate(k_folds):
    clf.fit(X.iloc[train], y.iloc[train])
    probs = clf.predict_proba(X.iloc[test])
    score = log_loss(y.iloc[test], probs)
    print score
    scores.append(score)
    
print(scores)
print("Average: " + str(np.average(scores)))

13.59359436
13.5791084898
13.5743742671
[13.593594360009373, 13.579108489795516, 13.574374267056021]
13.582359039


This submission function came from [this script](https://www.kaggle.com/shifanmao/sf-crime/random-forest-2/code)

We modified this slightly!

In [ ]:
def make_submission(clf, path='my_submission.csv'):
    '''This function will take in a model and a an optional 
    filepath and create a submissision file for us.'''
   
    test_data = pd.read_csv('test.csv')
    
    test_data, columnsToUse = recodeData(test_data)
    
    X_test_data = test_data[columnsToUse]
    ids = test_data['Id']
    
    #Predict the probabilities of each crime. 
    y_prob = clf.predict_proba(X_test_data)

    with open(path, 'w') as f:
        f.write("ID,")
        f.write(','.join(clf.classes_))
        f.write('\n')

        for id, probs in zip(ids, y_prob):
#            probas = ','.join([id] + list(map(str, probs.tolist())))
            probas = ','.join([str(id)] + map(str,["%.2f" % elem for elem in list(map(float,probs.tolist()))]))
#            probas = ','.join(list(map(str, probs.tolist())))
            f.write(probas)
            f.write('\n')
    print(" -- Wrote submission to file {}.".format(path))


In [ ]:
make_submission(clf)